In [1]:
print("all ok")

all ok


In [2]:
import os
from dotenv import load_dotenv

In [3]:
os.environ["HUGGING_FACE_KEY"]=os.getenv("HUGGING_FACE_KEY")

In [4]:
### Embeddings
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-l6-v2")

c:\Users\saxenanidhi\genai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
### Index
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
index=faiss.IndexFlatL2(384)

In [6]:
### Vectore store
vector_store= FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [7]:
doucments=["You can reset your password by clicking 'Forgot Password' on the login page.",
    "Our return policy allows returns within 30 days of purchase with a receipt.",
    "Shipping typically takes 3-5 business days for domestic orders."]

In [8]:
vector_store.add_texts(doucments)

['efb9b9a6-9f9f-4dad-8bc9-d8fffaac9ff4',
 '2e516e02-88da-4209-a483-949aad63f765',
 '08f06535-fbd2-4451-8a2c-52387d0648f3']

In [9]:
### Retriever
retriever=vector_store.as_retriever(search_kwargs={"k":3})

In [10]:
### LLM
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [11]:
### output parser
from langchain_core.output_parsers import StrOutputParser
outputParser =StrOutputParser()

In [12]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [13]:
### toxicity validation
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tox_model_name = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(tox_model_name)
tox_model = AutoModelForSequenceClassification.from_pretrained(tox_model_name)

def is_toxic(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        outputs = tox_model(**inputs)
        scores = torch.sigmoid(outputs.logits)[0].numpy()
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    result = dict(zip(labels, scores))
    toxic_score = result["toxic"]
    return toxic_score > 0.5  # You can adjust this threshold!

c:\Users\saxenanidhi\genai\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saxenanidhi\.cache\huggingface\hub\models--unitary--toxic-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
#### PROMPT template and RAG chain
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate
def function_1(state:AgentState):
    
    question=state["messages"][-1]
    print("-> RAG Call question->"+question)
    
    prompt=PromptTemplate(
        template="""
        You are a helpful customer support assistant. Use only the provided context to answer. 
        If you don't know the answer, say: 'Please connect with a support agent.'

        Context:
        {context}

        Question:
        {question}
        """,

        input_variables=['context', 'question']
    )
    
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    user_question = state["messages"][-1]
    if user_question.lower() in ["quit", "exit"]:
        return print("Bye")
    if is_toxic(user_question):
        print("AI Assistant: I'm here to help with respectful communication. Please rephrase your question politely.")
    else:
        result = rag_chain.invoke(question)
        return {"messages": [result]}

In [15]:
state={"messages":["i forgot my password.please help"]}

In [16]:
function_1(state)

-> RAG Call question->i forgot my password.please help


{'messages': ["You can reset your password by clicking 'Forgot Password' on the login page."]}

In [17]:
state={"messages":["you are so useless."]}

In [18]:
function_1(state)

-> RAG Call question->you are so useless.
AI Assistant: I'm here to help with respectful communication. Please rephrase your question politely.
